In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cuda is available.


## Loading the data

데이터에 존재하는 태그 파악하여 딕셔너리 생성

In [ ]:
# 임의로 태그 추가 : LN : Loan(대출), IN : Insurance(보험), AC : Account(계좌), CD : Card(카드), BN : Bank(은행), PB : Pass Book(통장), CR : Credit Score(신용점수)

tags = {'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-LC': 3, 'I-LC': 4,
        'B-OG': 5, 'I-OG': 6, 'B-QT': 7, 'I-QT': 8, 'B-DT': 9,
        'I-DT': 10, 'B-TI': 11, 'I-TI': 12, 'B-LT': 13, 'I-LT': 14,
        'B-PO': 15, 'I-PO': 16, 'B-LN' : 17, 'I-LN' : 18, 'B-IN' : 19,
        'I-IN' : 20, 'B-AC' : 21, 'I-AC' : 22, 'B-CD' : 23, 'I-CD': 24,
        'B-BN' : 25, 'I-BN' : 26, 'B-PB' : 27, 'I-PB' : 28, 'B-CR' : 29,
        'I-CR' : 30
        }
tag_size = len(tags)
tag_size

31

In [ ]:
print(tags) # <- tag_to_index

{'O': 0, 'B-PS': 1, 'I-PS': 2, 'B-LC': 3, 'I-LC': 4, 'B-OG': 5, 'I-OG': 6, 'B-QT': 7, 'I-QT': 8, 'B-DT': 9, 'I-DT': 10, 'B-TI': 11, 'I-TI': 12, 'B-LT': 13, 'I-LT': 14, 'B-PO': 15, 'I-PO': 16, 'B-LN': 17, 'I-LN': 18, 'B-IN': 19, 'I-IN': 20, 'B-AC': 21, 'I-AC': 22, 'B-CD': 23, 'I-CD': 24, 'B-BN': 25, 'I-BN': 26, 'B-PB': 27, 'I-PB': 28, 'B-CR': 29, 'I-CR': 30}


In [ ]:
index_to_tag = {j : i for i,j in tags.items()}
print(index_to_tag)

{0: 'O', 1: 'B-PS', 2: 'I-PS', 3: 'B-LC', 4: 'I-LC', 5: 'B-OG', 6: 'I-OG', 7: 'B-QT', 8: 'I-QT', 9: 'B-DT', 10: 'I-DT', 11: 'B-TI', 12: 'I-TI', 13: 'B-LT', 14: 'I-LT', 15: 'B-PO', 16: 'I-PO', 17: 'B-LN', 18: 'I-LN', 19: 'B-IN', 20: 'I-IN', 21: 'B-AC', 22: 'I-AC', 23: 'B-CD', 24: 'I-CD', 25: 'B-BN', 26: 'I-BN', 27: 'B-PB', 28: 'I-PB', 29: 'B-CR', 30: 'I-CR'}


load_data 함수를 정의 : 데이터를 불러와 문장, 태그(labels) 구분하여 저장

In [ ]:
def load_data(file_path):
  texts, labels = [], []

  with open(file_path, 'r', encoding = 'utf-8') as file:
    text, label = [], []
    for line in file:
      if line.startswith('##') or not line.strip():
        if text:
          texts.append(" ".join(text))
          labels.append([tags[i] for i in label])
          text, label = [], []
        continue
      parts = line.strip().split('\t')
      if len(parts) != 2:
        continue
      char, tag = parts
      text.append(char)
      label.append(tag)

    return texts, labels

train, dev data에서 각각 text, label을 구분하여 저장

In [ ]:
train_texts, train_labels = load_data('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Summer/NLP/Group Project/Phase 3/Data/klue_train.tsv')
dev_texts, dev_labels = load_data('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Summer/NLP/Group Project/Phase 3/Data/klue_dev.tsv')

In [ ]:
train_texts[1:3]

['한 군 데 서 필 름 을 너 무 낭 비 한 작 품 입 니 다 .', '하 지 만 이 영 화 에 는 감 히 별 5 개 를 주 고 싶 다']

In [ ]:
train_labels[1:3]

[[7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0]]

In [ ]:
print(f"train data는 {len(train_texts)}개 있고, dev data는 {len(dev_texts)}개 있답니다!")

train data는 21008개 있고, dev data는 5000개 있답니다!


## 네이버 지식 iN 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup


questions = []
for i in range(1,3):
  # 질문 목록 페이지 URL
  list_url = f"https://kin.naver.com/qna/list.naver?dirId=401&queryTime=2024-09-18%2013%3A25%3A56&page={i}"

  # 질문 목록 페이지에서 각 질문의 URL들 추출
  r = requests.get(list_url)
  soup = BeautifulSoup(r.text, "html.parser")

  # 각 url에서 <a 부분이 각 질문에 따라 달라지는 부분 -> 그 부분을 추출
  question_links = soup.select("td.title > a")
  # print("첫 번째 url:", question_links[0])

  # 기본 url + 위에서 뽑아낸 부분에서 'href'부분 합쳐서 질문별 url 생성
  base_url = "https://kin.naver.com"
  question_urls = [base_url + link['href'] for link in question_links]

  # 2. 각 질문 URL에서 질문 내용을 추출
  for i, question_url in enumerate(question_urls[:50],1):
      r = requests.get(question_url)
      soup = BeautifulSoup(r.text, "html.parser")

      # 질문 내용이 있는 부분 선택
      text = soup.select_one(".questionDetail").text.strip()
      questions.append(text)

In [ ]:
len(questions)

40

In [ ]:
naver_texts = []
naver_labels = []
for question in questions:
  # 한글만 남기는 작업 진행
  question = re.sub(r"[^가-힣]","",question)
  wo_blank = question.replace(' ','') # 공백제거
  num = [0] * len(wo_blank)
  result = ' '.join(list(wo_blank))
  naver_labels.append(num)
  naver_texts.append(result)

In [ ]:
naver_texts[1:3]

['상 품 이 아 닌 인 도 채 권 을 구 매 하 려 면 어 떻 게 해 야 하 나 요 미 국 채 권 처 럼 구 매 대 행 을 해 주 는 금 융 사 가 있 으 면 알 려 주 세 요',
 '제 가 앱 으 로 주 택 청 약 가 입 했 었 는 데 사 정 이 생 겨 서 해 지 하 려 고 합 니 다 근 데 앱 으 로 해 지 가 안 되 는 거 같 은 데 고 객 센 터 전 화 해 서 해 지 가 능 한 가 요']

In [ ]:
naver_labels[1:3]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

## 은행 FAQ 질문들

In [ ]:
questions = []
url1 = "https://www.kebhana.com/cont/customer/customer01/index.jsp"
r = requests.get(url1)
soup = BeautifulSoup(r.text, "html.parser")
items = soup.select("ul.faqList li a")

for item in items:
  text = item.get_text(strip = True)[4:]
  text = re.sub(r"[^가-힣]","",text)
  questions.append(text)

In [ ]:
def get_questions(url):

  r = requests.get(url)

  soup = BeautifulSoup(r.text, "html.parser")

  # .questionDetail 요소 선택
  items = soup.select("ul.faqList li a")

  # 리스트로 반환된 각 요소에서 텍스트 추출
  for item in items:
      text = item.get_text(strip = True)[4:]
      text = re.sub(r"[^가-힣]","",text)
      questions.append(text)

In [ ]:
for i in range(2,5,2):
  url = f"https://www.kebhana.com/cont/customer/customer01/index,1,list,{i}.jsp"
  get_questions(url)

In [ ]:
faq_texts = []
faq_labels = []
for question in questions:
  wo_blank = question.replace(' ','') # 공백제거
  num = [0] * len(wo_blank)
  result = ' '.join(list(wo_blank))
  faq_labels.append(num)
  faq_texts.append(result)

In [ ]:
faq_texts[1:3]

['여 권 조 합 번 호 를 실 명 번 호 로 사 용 하 고 있 습 니 다 인 증 서 를 발 급 받 을 수 있 나 요',
 '금 융 인 증 서 비 밀 번 호 를 분 실 한 경 우 어 떻 게 해 야 하 나 요']

In [ ]:
faq_labels[1:3]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## Tagging 진행

In [ ]:
tagged_texts = ['현 재 나 이 4 0 대 일 용 직 이 며 수 입 이 2 0 0 선 입 니 다',
                '저 희 아 빠 가 7 억 을 빚 져 서 코 인 하 다 잃 었 는 데 개 인 회 생 못 하 나 요',
                '삼 성 전 자 주 식 하 락 이 유 가 궁 금 하 네 요',
                '올 리 브 영 결 제 할 때 계 좌 로 돈 보 내 서 결 제 하 는 거 되 나 요',
                '주 말 추 석 우 체 국 문 여 나 요',
                '토 일 월 화 수 다 안 열 면 목 요 일 에 만 수 령 할 수 있 다 는 건 데',
                '미 국 에 서 1 0 0 0 달 러 상 품 을 직 구 하 려 는 데',
                '신 생 아 특 례 대 출 에 해 당 하 는 조 건 들 만 맞 으 면 대 출 이 무 조 건 나 오 는 건 가 요 ?',
                '제 가 삼 성 화 재 에 9 월 1 2 일 날 가 입 을 하 고 그 날 보 험 료 납 입 을 했 습 니 다 .',
                '청 년 주 택 드 림 청 약 을 가 입 하 려 고 합 니 다 .',
                '미 성 년 자 도 카 카 오 핸 드 폰 보 험 가 능 한 가 요 ?',
                '카 카 오 뱅 크 비 상 금 대 출 연 장 가 능 할 까 요 ?',
                '작 년 에 한 번 연 장 했 고 이 번 에 도 연 장 하 고 싶 은 데',
                '연 체 이 력 은 없 고 하 나 은 행 비 상 금 대 출 3 0 0 이 추 가 로 있 습 니 다',
                '신 용 점 수 는 K C B - 7 6 2 , N I C E - 8 3 9 인 데 연 장 가 능 할 까 요 ?'
                ]

tagged_labels = [[0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 0, 0, 0],
                 [0, 0, 1, 2, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [5, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [5, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [9, 10, 10, 10, 13, 14, 14, 0, 0, 0, 0],
                 [9, 10, 10, 10, 10, 0, 0, 0, 0, 9, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [3, 4, 0, 0, 8, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [15, 16, 16, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 0, 19, 20, 20, 20, 0, 7, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [27, 28, 28, 28, 28, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [1, 2, 2, 2, 0, 25, 26, 26, 19, 20, 20, 20, 20, 0, 0, 0, 0, 0, 0],
                 [26, 27, 27, 27, 27, 17, 18, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0],
                 [9, 10, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 17, 18, 18, 18, 18, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0],
                 [29, 30, 30, 30, 0, 29, 30, 30, 0, 7, 8, 8, 0, 29, 30, 30, 30, 7, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                 ]

############### tagged_labels = [[str(item) for item in items] for items in tagged_labels] #########

In [ ]:
tagged_texts[1:3]

['저 희 아 빠 가 7 억 을 빚 져 서 코 인 하 다 잃 었 는 데 개 인 회 생 못 하 나 요',
 '삼 성 전 자 주 식 하 락 이 유 가 궁 금 하 네 요']

In [ ]:
tagged_labels[1:3]

[[0,
  0,
  1,
  2,
  0,
  7,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [5, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

## [***추가***] 금융 관련 fake_data 학습

카드 - 신용카드, 체크카드, 교통카드, 선불카드, 현금카드, 트래블릿카드,
은행 - 국민, 신한, 하나, 우리, 농협, 토스, 카카오,
계좌 - 주식거래계좌, 외화계좌, 저축계좌, 보통예금계좌, ISA, 펀드계좌, 정기예금계좌, 채권계좌,
통장 - 적금통장, 예금통장, 일반통장, 급여통장, 파킹통장, 주택청약종합저축

In [ ]:
import random

def generate_fake_data_train(num_samples):
  banks = ["국 민 은 행", "신 한 은 행", "하 나 은 행", "우 리 은 행", "농 협 은 행", "토 스 증 권"]
  cards = ["신 용 카 드", "체 크 카 드", "교 통 카 드", "선 불 카 드", "현 금 카 드", "트 래 블 릿"]
  accounts = ["주 식 계 좌", "외 화 계 좌", "저 축 계 좌", "예 금 계 좌", "펀 드 계 좌", "채 권 계 좌"]
  passbooks = ["적 금 통 장", "예 금 통 장", "일 반 통 장", "급 여 통 장", "파 킹 통 장"]
  insurances = ["생 명 보 험", "손 해 보 험", "연 금 보 험", "건 강 보 험", "화 재 보 험"]
  loans = ["담 보 대 출", "전 세 대 출", "신 용 대 출", "기 업 대 출", "개 인 대 출", "청 년 대 출"]

  structures = [
      lambda: f"{bank} 의 {card} 와 {account} , {passbook} 을 사 용 하 며 {insurance} 와 {loan} 이 있 습 니 다",
      lambda: f"{insurance} 와 {loan} 이 있 고 , 주 거 래 은 행 은 {bank} 이 며 {account} 와 {passbook}, 그 리 고 연 결 된 {card} 를 사 용 합 니 다 .",
      lambda: f"{loan} 을 받 은 상 태 이 며 , {bank} 의 {account} 와 {card} 를 사 용 중 . {insurance} 는 {passbook} 과 연 계 되 어 있 습 니 다 .",
      lambda: f"{insurance} 와 {loan} 을 보 유 하 고 , 주 요 계 좌 는 {bank} 에 있 으 며 {account} 와 {card} 를 활 용 합 니 다 . 추 가 로 {passbook} 을 사 용 중 입 니 다.",
      lambda: f"{loan} 을 받 고 , 주 계 좌 는 {bank} 이 며 {passbook} 을 활 용 중 입 니 다 . {insurance} 와 {card} 도 사 용 합 니 다 .",
      lambda: f"{bank} 의 {account} 를 사 용 하 고 , {passbook} 과 {insurance} 를 연 결 했 습 니 다 . 또 한 , {loan} 와 {card} 도 보 유 중 입 니 다 .",
      lambda: f"주 은 행 은 {bank} 이 며 , {loan} 와 {passbook} 을 사 용 중 입 니 다 . 그 리 고 {insurance} 와 {account} 를 활 용 하 고 있 습 니 다 .",
      lambda: f"{insurance} 와 {loan} 을 보 유 하 고 , 주 요 계 좌 는 {bank} 에 있 으 며 {account} 와 {passbook} 을 사 용 중 입 니 다 . 또 한 {card} 도 있 습 니 다 .",
      lambda: f"주 은 행 은 {bank} 이 며 , {card} 와 {loan} 을 활 용 하 고 , {insurance} 와 {account} 도 사 용 중 입 니 다 . 추 가 로 {passbook} 이 있 습 니 다 .",
      lambda: f"{card} 와 {loan} 을 사 용 중 이 며 , {passbook} 와 {account} 도 보 유 중 입 니 다 . 그 리 고 주 요 은 행 은 {bank} 이 며 {insurance} 를 사 용 중 입 니 다 ."
  ]

  labels = [
      [23, 24, 23, 24, 0, 25, 26, 26, 26, 0, 21, 22, 22, 22, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0],
      [19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 21, 22, 22, 22, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0],
      [17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 21, 22, 22, 22, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      [19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 0, 0, 21, 22, 22, 22, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0],
      [17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0],
      [23, 24, 24, 24, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 27, 28, 28, 28, 0, 19, 20, 20, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 18, 18, 18, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 17, 18, 18, 18, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      [19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 0, 0, 21, 22, 22, 22, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 25, 26, 26, 26, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0],
      [25, 26, 26, 26, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 27, 28, 28, 28, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 19, 20, 20, 20, 0, 0, 0, 0, 0, 0, 0, 0]
  ]

  data = []
  tags = []

  for _ in range(num_samples):
    bank = random.choice(banks)
    card = random.choice(cards)
    account = random.choice(accounts)
    passbook = random.choice(passbooks)
    insurance = random.choice(insurances)
    loan = random.choice(loans)

    i = random.randint(0, len(structures)-1)

    structure = structures[i]()
    label = labels[i]

    data.append([structure, label])

  return data

In [ ]:
num_samples = 10000
fake_data = generate_fake_data_train(num_samples)

fin_texts_train = []
fin_labels_train = []

for i in range(num_samples):
  sent, tag = fake_data[i]
  fin_texts_train.append(sent)
  fin_labels_train.append(tag)

len(fin_texts_train)

10000

In [ ]:
import random

def generate_fake_data_dev(num_samples):
  banks = ["국 민 은 행", "신 한 은 행", "하 나 은 행", "우 리 은 행", "농 협 은 행", "토 스 증 권"]
  cards = ["신 용 카 드", "체 크 카 드", "교 통 카 드", "선 불 카 드", "현 금 카 드", "트 래 블 릿"]
  accounts = ["주 식 계 좌", "외 화 계 좌", "저 축 계 좌", "예 금 계 좌", "펀 드 계 좌", "채 권 계 좌"]
  passbooks = ["적 금 통 장", "예 금 통 장", "일 반 통 장", "급 여 통 장", "파 킹 통 장"]
  insurances = ["생 명 보 험", "손 해 보 험", "연 금 보 험", "건 강 보 험", "화 재 보 험"]
  loans = ["담 보 대 출", "전 세 대 출", "신 용 대 출", "기 업 대 출", "개 인 대 출", "청 년 대 출"]

  structures = [
      lambda: f"{passbook} 을 사 용 하 며 , {bank} 의 {card} 와 {loan} 을 활 용 중 입 니 다 . 또 한 , {insurance} 와 {account} 도 연 결 되 어 있 습 니 다 .",
      lambda: f"주 은 행 은 {bank} 이 며 , {loan} 와 {insurance} 를 활 용 중 입 니 다 . 그 리 고 {card} 와 {passbook}, {account} 도 보 유 중 입 니 다 .",
      lambda: f"{account} 와 {passbook} 을 주 로 사 용 하 며 , {insurance} 와 {loan} 도 연 결 되 어 있 습 니 다 . 그 리 고 {bank} 의 {card} 도 있 습 니 다 .",
      lambda: f"{insurance} 와 {loan} 을 주 로 사 용 중 이 며 , 주 요 은 행 은 {bank} 이 며 {account}, {passbook} 과 연 결 된 {card} 를 활 용 하 고 있 습 니 다 .",

  ]

  labels = [
      [27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 25, 26, 26, 26, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      [0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 17, 18, 18, 18, 0, 19, 20, 20, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 26, 0, 27, 28, 28, 28, 0, 21, 22, 22, 22, 0, 0, 0, 0, 0, 0, 0, 0],
      [21, 22, 22, 22, 0, 27, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0],
      [19, 20, 20, 20, 0, 17, 18, 18, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 24, 24, 24, 0, 0, 21, 22, 22, 22, 0, 27, 28, 28, 28, 0, 0, 0, 0, 25, 26, 26, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  ]

  data = []
  tags = []

  for _ in range(num_samples):
    bank = random.choice(banks)
    card = random.choice(cards)
    account = random.choice(accounts)
    passbook = random.choice(passbooks)
    insurance = random.choice(insurances)
    loan = random.choice(loans)

    i = random.randint(0, len(structures)-1)

    structure = structures[i]()
    label = labels[i]

    data.append([structure, label])

  return data



In [ ]:
num_samples = 2000
fake_data = generate_fake_data_dev(num_samples)

fin_texts_dev = []
fin_labels_dev = []

for i in range(num_samples):
  sent, tag = fake_data[i]
  fin_texts_dev.append(sent)
  fin_labels_dev.append(tag)

len(fin_texts_dev)

2000

## [***변경***] 합치기!

길이 맞는지 체크해보고 넣자

In [ ]:
print(len(train_texts) , len(naver_texts), len(faq_texts), len(tagged_texts), len(fin_texts_train))
print(len(train_labels), len(naver_labels), len(faq_labels), len(tagged_labels), len(fin_labels_train))

21008 40 31 15 10000
21008 40 31 15 10000


In [ ]:
naver_texts = [str(item) for item in naver_texts]
faq_texts = [str(item) for item in faq_texts]
tagged_texts = [str(item) for item in tagged_texts]
fin_texts = [str(item) for item in fin_texts_train]

In [ ]:
train_texts = train_texts + naver_texts + faq_texts + tagged_texts + fin_texts

In [ ]:
naver_labels = [[int(item) for item in items] for items in naver_labels]
faq_labels = [[int(item) for item in items] for items in faq_labels]
tagged_labels = [[int(item) for item in items] for items in tagged_labels]
fin_labels = [[int(item) for item in items] for items in fin_labels_train]

In [ ]:
train_labels = train_labels + naver_labels + faq_labels + tagged_labels + fin_labels

In [ ]:
for index, items in enumerate(train_labels):
  for item in items:
    if not isinstance(item, int):
      print(index, items)

In [ ]:
fin_texts_dev = [str(item) for item in fin_texts_dev]
fin_labels_dev = [[int(item) for item in items] for items in fin_labels_dev]

In [ ]:
dev_texts = dev_texts + fin_texts_dev
dev_labels = dev_labels + fin_labels_dev

## Tokenize, Padding

In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader

In [ ]:
class PIIDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]


        encoding = self.tokenizer.encode_plus(text,
                                              add_special_tokens = True,
                                              max_length = self.max_length,
                                              return_token_type_ids = False,
                                              padding = 'max_length',
                                              truncation = True,
                                              return_attention_mask = True,
                                              return_tensors = 'pt'
                                              )

        tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

        temp_labels = [0] * len(tokens)

        j = 0
        for i, token in enumerate(tokens):
          if token.startswith("##"):
            continue
          if token in ['[CLS]','[SEP]','[PAD]']:
            temp_labels[i] = 0
          elif j < len(label):
            temp_labels[i] = label[j]
            j += 1

        return {
            'input_ids' : encoding['input_ids'].flatten(),
            'attention_mask' : encoding['attention_mask'].flatten(),
            'labels' : torch.tensor(temp_labels, dtype = torch.long)
        }

## Dataset, DataLoader

In [ ]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
max_length = 128

# Dataset
train_data = PIIDataset(train_texts, train_labels, tokenizer, max_length)
dev_data = PIIDataset(dev_texts, dev_labels, tokenizer, max_length)

# DataLoader
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle = True)
dev_dataloader = DataLoader(dev_data, batch_size = 32, shuffle = False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

model = BertForTokenClassification.from_pretrained('klue/bert-base', num_labels=tag_size)
model.to(device)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Model Fitting

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 1e-5, # 기존 2e-5에서 수정
    per_device_train_batch_size = 16, # 기존 32에서 수정
    per_device_eval_batch_size = 16, # 기존 32에서 수정
    num_train_epochs = 5, # 기존 5에서 수정
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    greater_is_better = False
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience = 3,
    early_stopping_threshold = 0.0
)

trainer = Trainer(
    model = model,
    args = training_args,
    callbacks = [early_stopping_callback],
    train_dataset = train_data,
    eval_dataset = dev_data
)

trainer.train()
trainer.evaluate()

model.save_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Summer/NLP/Group Project/Phase 5/')
tokenizer.save_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Summer/NLP/Group Project/Phase 5/')

print(" 모델 저장 완료! :D ")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.039300,0.084864
2,0.037700,0.077607
3,0.021200,0.074679


Epoch,Training Loss,Validation Loss
1,0.039300,0.084864
2,0.037700,0.077607
3,0.021200,0.074679
4,0.014600,0.079862
5,0.014900,0.077948


 모델 저장 완료! :D 


## Loading the Model

In [1]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2024 Summer/NLP/Group Project/Phase 5/')
model.to(device)

NameError: name 'device' is not defined

## Functions for new Sentences

In [ ]:
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

### ner_pii_masking

----------------
< ner_pii_masking >
- 가장 기본이 되는 함수
- 모든 label 마스킹, [MASK] 로 출력
----------------

In [ ]:
def ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += (f"[MASK-{tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (f"[MASK-{tag_name}]")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(num[pad_count])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'이비비는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.'

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 [MASK-PS]는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.'

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 농협 1040-452-394027 은행을 쓴다. 저축계좌가 있으며, 적금통장도 있다!"
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 [MASK-PS]는 우리 집 막내이며, [MASK-LC] [MASK-LC]에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 농협 [ACC] 은행을 쓴다. 저축계좌가 있으며, 적금통장도 있다 !'

In [ ]:
sentence = "네, 강지은 고객님 본인 맞으실까요? 주소는 현재 서울특별시 강남구로 되어 있습니다. 저희 농협은행에 건강보험 문의 주셨어요. 현재 1002-123-123456 계좌 쓰고 계시구요, 전화번호는 010-1234-5678 맞으시죠? 적금통장 만들고 싶으시다구요. "
ner_pii_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'네, [MASK-PS] 고객님 본인 맞으실까요 ? 주소는 현재 [MASK-LC] [MASK-LC]로 되어 있습니다. 저희 [MASK-OG]에 건강보험 문의 주셨어요. 현재 [ACC] 계좌 쓰고 계시구요, 전화번호는 [TEL] 맞으시죠 ? 적금통장 만들고 싶으시다구요.'

### ner_pii_masking_loose

----------------
< ner_pii_masking_loose >
- 루즈한 마스킹 모델
- 이름, 계좌번호, 주민등록번호, 전화번호 마스킹
----------------

In [ ]:
def ner_pii_masking_loose(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      tag_name = index_to_tag[tag_index][2:]

      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif tag_name == 'PS':
        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += (f"[MASK-{tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (f"[MASK-{tag_name}]")

      else: # 가릴 게 없다면
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)



      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(num[pad_count])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking_loose(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 [MASK-PS]는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 [TEL] 이고, 주민등록번호는 [RRN] 이다. 또한, 우리 [ACC] 은행을 쓴다.'

### ner_pii_masking_w_ast

----------------
< ner_pii_masking_w_ast >
- 다른 방식으로 마스킹 진행
- 모든 label 마스킹, * 로 출력
- 전화번호, 주민등록번호, 계좌번호의 경우 자릿수 맞춰서 asterisk로 표현 0
----------------

In [ ]:
def ner_pii_masking_w_ast(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : '[RRN]',

      r'\d{3}-\d{2}-\d{6,7}' : '[ACC]',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : '[ACC]',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : '[ACC]',
      r'\d{3}-\d{6}-\d{5}' : '[ACC]',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : '[ACC]',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : '[ACC]',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : '[ACC]',
      r'\d{4}-\d{2,3}-\d{6,7}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}' : '[ACC]',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : '[ACC]',
      r'\d{1}-\d{6}-\d{2,3}' : '[ACC]',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : '[ACC]',

      r'\d{3}-\d{4}-\d{4}' : '[TEL]',
      r'\d{2}-\d{3,4}-\d{4}' : '[TEL]'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None
  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):

        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            current_token += ("***")

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += ("***")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  ast = {'[TEL]':'***-****-****', '[RRN]':'******-*******', '[ACC]':'***-**-******'}
  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(ast[num[pad_count]])
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
ner_pii_masking_w_ast(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 ***는 우리 집 막내이며, *** 강남구에 살고 있다. 전화번호는 ***-****-**** 이고, 주민등록번호는 ******-******* 이다. 또한, 우리 ***-**-****** 은행을 쓴다.'

### identifier_masking

----------------
< identifier_masking >
- 전화번호, 주민등록번호, 이름 의 경우 식별자로 표현
- 나머지는 식별 가능 정보
----------------

In [ ]:
def identifier_masking(sentence, model, tokenizer, index_to_tag, max_length, device):

  # 숫자 정보 마스킹 후 리스트 num에 저장 (순서대로) ---------------------------
  numbers = {
      r'\d{6}-\d{7}' : 'RRN',

      r'\d{3}-\d{2}-\d{6,7}' : 'ACC',
      r'\d{3}-\d{3}-(\d{6}|\d{8})' : 'ACC',
      r'\d{3}-(\d{5}|\d{8})-\d{3}' : 'ACC',
      r'\d{3}-\d{6}-\d{5}' : 'ACC',
      r'\d{3}-\d{4}-\d{4}-\d{2}' : 'ACC',
      r'\d{3}-\d{5}-\d{3}-\d{2}' : 'ACC',
      r'\d{3}-\d{6}-\d{2}-\d{3}' : 'ACC',
      r'\d{4}-\d{2,3}-\d{6,7}' : 'ACC',
      r'\d{4}-\d{4}-\d{4}' : 'ACC',
      r'\d{4}-\d{4}-\d{4}-\d{1}' : 'ACC',
      r'\d{1}-\d{6}-\d{2,3}' : 'ACC',
      r'(\d{2}|\d{6})-\d{2}-\d{6}' : 'ACC',

      r'\d{3}-\d{4}-\d{4}' : 'TEL',
      r'\d{2}-\d{3,4}-\d{4}' : 'TEL'
  }

  sorted_patterns = sorted(numbers.items(), key=lambda x: len(x[0]), reverse=True)

  num = []
  matches = []
  already_matched = set()

  for pattern, token in sorted_patterns:
    for match in re.finditer(pattern, sentence):
      start = match.start()
      end = match.end()
      if all(pos not in already_matched for pos in range(start, end)):
        matches.append((match.start(), token))
        already_matched.update(range(start, end))


  matches.sort(key = lambda x : x[0])

  num = [token for _, token in matches]

  # 숫자 정보를 제외한 문자 정보만으로 input 만든 뒤 모델에 넣어 ---------------

  pattern = '|'.join(f'({key})' for key in numbers.keys())

  sentence_no_num = re.sub(pattern, '[PAD]', sentence) # 숫자 정보는 [PAD] 으로 처리

  tokenized_sentence = tokenizer.tokenize(sentence_no_num)

  sent_in_num = tokenizer.convert_tokens_to_ids(tokenized_sentence)

  input_sentence = pad_sequences([sent_in_num], maxlen = max_length, dtype = 'int',
                                    value = 0, truncating = 'post', padding = 'post')

  input_mask = [[1 if tok > 0 else 0 for tok in sent] for sent in input_sentence]

  input_sentence = torch.tensor(input_sentence).to(device)
  input_mask = torch.tensor(input_mask).to(device)

  with torch.no_grad():
    output = model(input_sentence, attention_mask = input_mask)
    logits = output.logits

  pred = np.argmax(logits.cpu().numpy(), axis = 2)

  tokens = tokenizer.convert_ids_to_tokens(input_sentence[0])

  # 다시 문장의 형태로 결합 ----------------------------------------------------

  out_sentence = []
  current_token = ""
  prev_tag = None
  tag_name = None

  puncs = [".",","]

  for token, tag_index in zip(tokens, pred[0]):
      if (token == '[PAD]'):
        if current_token:
          out_sentence.append(current_token)
          current_token = ""
        out_sentence.append('[PAD]')

      elif (tag_index == 0):
        if token in puncs:
          current_token += token

        elif token.startswith("##"):
          current_token += (token[2:])

        else:
          out_sentence.append(current_token)
          current_token = ""
          current_token += (token)

      elif (tag_index > 0):
        tag_name = index_to_tag[tag_index][2:]

        if token.startswith("##"):
          if tag_name == prev_tag:
            pass
          else:
            if tag_index in [1,2]:
              current_token += (f"[식별자 - {tag_name}]")
            else:
              current_token += (f"[개인식별가능정보 - {tag_name}]")

        else:
          out_sentence.append(current_token)
          current_token = ""
          if tag_index in [1,2]:
            current_token += (f"[식별자 - {tag_name}]")
          else:
            current_token += (f"[개인식별가능정보 - {tag_name}]")

      prev_tag = tag_name

  out_sentence.append(current_token)

  out_sentence = ' '.join(out_sentence)

  # [PAD] -> 마스킹 토큰으로 ---------------------------------------------------

  out_sentence_split = out_sentence.split()

  new_sentence = []

  pad_count = 0

  for token in out_sentence_split:
    if token == '[PAD]':
      if pad_count < len(num):
        new_sentence.append(f"[식별자 - {num[pad_count]}]")
        pad_count += 1
    else:
      new_sentence.append(token)

  out_sentence = ' '.join(new_sentence)

  return(out_sentence)

In [ ]:
sentence = "고양이 이비비는 우리 집 막내이며, 서울 강남구에 살고 있다. 전화번호는 010-2453-4063이고, 주민등록번호는 180815-3045928이다. 또한, 우리 1040-452-394027 은행을 쓴다."
identifier_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'고양이 이비[식별자 - PS]는 우리 집 막내이며, [개인식별가능정보 - LC] 강남구에 살고 있다. 전화번호는 [식별자 - TEL] 이고, 주민등록번호는 [식별자 - RRN] 이다. 또한, 우리 [식별자 - ACC] 은행을 쓴다.'

In [ ]:
sentence = '''행복한 농협 곽수현 상담원입니다.

아 혹시 체크카드 해지 관련해서 여쭤봐도 될까요?

네 해지하시려고요.

네 아 근데 일단 그 전에 잔액 이 체크카드가 어떤 계좌랑 연결돼 있는지 제가 지금 모르겠어서 카드랑 연결된 계좌가 어떤 계좌인지 확인, 또 잔액 조회 이런 것도 가능할까요?

계좌는 카드 정보 확인되시면 안내가 가능하고요. 잔액은 은행 쪽으로 문의해 보셔야 되시고 일단 계좌번호 말씀을 드릴 건데 사용하시는 그 카드 번호를 말씀해 주시겠습니까?

5461 3620 3802

네, 강지은 고객님 본인 맞습니까? 멘트 따라서 4자리 카드 비밀번호 확인하겠습니다. 비밀번호 4자리를 입력해 주십시오.

네 말씀해주신 카드와 연결된 계좌는 농협은행 302 0079233911계좌입니다. 그 계좌의 잔액은 은행 쪽으로 1661-3000으로 문의해 보시면 되시구요.

관련 없을 수도 있는데, 제가 이거 만들 때 외화계좌를 만들어서 해외 결제용으로 만들었었는데요. 근데 방금 계좌는 제 국내에 그 계좌라 뭘 혹시 맞는지 확인하려면 어떻게 해야 될까요?

이 계좌가 외화계좌인지 아닌지 구분은 어려운데요. 지금 말씀하신 카드가 트래블릿카드처럼 외화계좌가 꼭 필요한 그 체크카드가 아니고 일반 체크카드거든요. 근데 일반 체크카드는 보통 국내 그 저기 원화 계좌로 보통 연결이 되는데 뭐 정확하게 확인하시려면 은행 콜센터 1661에 3천번 쪽으로 문의하셔야 됩니다. 고객님 그리고 거기서 이제 확인하시고 카드가 필요 없으시면 인제 여기를 통해서 해지를 하셔야 되시는 거구요.

아 네, 알겠습니다.

네 1661-3000번 하셔서 예금업무 1번 누르시고요. 상담사 매뉴얼 이용하시기 바랍니다.

네 감사합니다.

네 감사합니다. 곽수현이었습니다.'''

identifier_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'행복한 농협 [식별자 - PS] 상담원입니다. 아 혹시 체크카드 해지 관련해서 여쭤봐도 될까요 ? 네 해지하시려고요. 네 아 근데 일단 그 전에 잔액 이 체크카드가 어떤 계좌랑 연결돼 있는지 제가 지금 모르겠어서 카드랑 연결된 계좌가 어떤 계좌인지 확인, 또 잔액 조회 이런 것도 가능할까요 ? 계좌는 카드 정보 확인되시면 안내가 가능하고요. 잔액은 은행 쪽으로 문의해 보셔야 되시고 일단 계좌번호 말씀을 드릴 건데 사용하시는 그 카드 번호를 말씀해 주시겠습니까 ? 5461 3620 3802 네,'

In [ ]:
sentence = "안녕하세요, 서울특별시 강남구에 사는 박준희입니다. 현재 농협 1000-000-000000을 사용 중이고, 주식계좌를 가지고 있습니다. 대출 "
identifier_masking(sentence, model, tokenizer, index_to_tag, max_length, device)

'안녕하세요, [개인식별가능정보 - LC] 강남구에 사는 [식별자 - PS]입니다. 현재 농협 [식별자 - ACC] 을 사용 중이고, 주식계좌를 가지고 있습니다. 대출'